## Partitions

Partition data is an important part of Spark SQL since it powers one of the key optimizations to allow reading only the required data, discussed more in Logical and Physical Plans on page 69. If you know how your downstream consumers may access your data, when you write your data it is beneficial to use that information to partition your output. When reading the data, it's useful to understand how parititon discovery functions, so you can have a better understanding of whether your filter can be pushed down/

When reading partitioned data, you point Spark to the root path of your data, and it will automatically discover the different partitions. Not all data types can be used as partition keys; currently only strings and numeric data are the supported types.

If your data is all in a single DataFrame, the DataFrameWriter API makes it easy to specify the partition information while you are writing the data out. The partitionBy function takes a list of columns to partition the ouput on, as shown in Example 3-49. You can also manually save out separate DataFrames with individual save calls.

## Datasets

Datasets are an exciting extension of Spark SQL that provide additional comile time type checking. Starting in Spark 2.0, DataFrames are not a specialized version of Datasets that operate on generic Row objects and therefore lack the normal compile-time type checking of Datasets. Datasets can be used when your data can be encoded for Spark SQL and you know the type information at compile time. The Dataset API is a strongly typed collection with a mixture of relational and functional transformations. Like DataFrames, Datasets are represented by a logical plan the Catalyst optimizer can work with, and when cached the data is stored in Spark SQL's internal encoding format.

## Interoperability with RDDs, DataFrames, and Local Collections

Datasets can be easily converted to/from DataFrames and RDDs, but in the initial version they do not directly extend either. Converting to/from RDDs involves encoding/decoding the data into a different form. Converting to/from DataFrames is almost free in that the underlying data does not need to be changed; only extra compile-time type information is added/removed/

## Compile-Time String Typing 

One of the reasons to use Datasets over traditional DataFrmaes is their compile-time strong typing. DataFrames have runtime schema information but lakck compile-time information about the schema. This strong typing is especially useful when making libraries, because you can more clearly specify the requirements of your inpus and your return types.

## Easier Functional Transformations

One of the key advantages of the Dataset API is easier integration with custom Scala and Java coe. Datasets expose filter, map, mapPartitions, and flatMap with similar function signatures as RDDs, with the notable requirement that your return ElementType also be understandable by Spark SQL.

## Relational Transformations

Datasets introduce a typed version of select for relational-style transformations. When specifying an expression for this you need to include the type information, as shown in Example 3-53.

## Grouped Operations on Datasets

Similar to grouped operations on DataFrames, groupBy on Datasets prior to Spark 2.0 returns a GroupedDataset or a KeyValueGroupedDataset when grouped with an arbitrary function, and a RelationalGroupedDataset when grouped with a relational/Dataset DSL expression. You can specify your aggregate functinos on all of these, along with a functional mapGroups API. As with the expression in Relational Transformations on page 64, you need to use typedexpressions so the result can also be a Dataset.

## Choosing a Join Type

The default join operation in Spark includes only alues for keys present in both RDDs, and in the case of multiple values per key, provides all permutations of the key/value pair. The best scenario for a standard join is when both RDDs contain the same set of distinct keys.

### Choosing an Execution Plan

In order to join data, Spark needs the data that is to be joined to live on the same partition. The default implementation of a join in Spark is a shuffled hash join. The shuffled hash join ensures that data on each  partition will contain the same keys y partitioning the second dataset with teh same default partitioner as the first, so that the keys with the same hash value from both datasets are in the same partition. Whiile this approach  always works, it can be more expensive than necessary because it requires a shuffle. The shuffle can be avoided if:

1. Both RDDs have a known partitioner.
2. One of the datasets is small enough to fit in memory, in which case we can do a broadcast hash join.

Note that if the RDDs are colocated the network transfer can be avoided, along with the shuffle.

#### Speeding up joins by assigning a known partitioner 
If you have to do an operation before the join that requires a shuffle, such as aggregateByKEy or reduceByKey, you can prevent the shuffle by adding a hash partitioner with the same number of partitions as an explicit argument to the first operation and persisting the RDD before the join. You could make the example in the previous section even faster, by using the partitioner for the address data as an argument for the reduceByKey step, as in Example 4-4 and Figure 4-4.

#### Speedingup joins using a broadcast hash join

A broadcast hash join pushes one of the RDDs to each of the worker nodes. Then it does a map-side combine with each partition of the larger RDD. If one of your RDDs can fit in memory or can be made to fit in memory it is always beneficial to do a broadcast hash join, since it doesn't require a shuffle. Sometimes Spark SQL will be smart enough to confiture the broadcast join itself; in Spark SQL this is controlled with spark.sql.autoBroadcastJOinThreshold and spark.sql.boradcastTimeout. This is illustrated in Figure 4-5.

## Spark SQL JOins

Spark SQL supports the same basic join types as core Spark, but the optimizer is ablt to do more of the heavy lifting for you - although you also give up some of your control. For example, Spark SQL can somtimes  push down or reorder operations to make your joins more efficient. On the other hand, you don't control the partitioner for DataFrames or Datasets, so you can't manually avoid shuffles as you did with core Spark joins.

### DataFrame Joins

Joining data between DataFrames is one of the most common multi-DataFrame transformations. the standard SQL join types are all supported and can be specified as the joinType id df.join when performing a join.

# Effective Transformations

## Narrow vs Wide Transformations

## Implications for Performance

In Wide vs Narrow Dependencies on page 17 we asserted that transformations with narrow dependencies are faster to execute partly because narrow transformations can be combined and executed in one pass of the data. In this section we hope explain why this is from an evaluation perspective.

Narrow dependencies do not require data to be moved across partitions. Consequently narrow transformations don't require communication with the driver node, and an arbitrary number of narrow transformations can be executed on any subset of the records given one set of instructions from the driver. In Spark terminology, we  say that each series of narrow transformations can be computed in the same stage of the query execution plan.

In contrast, as we stated in The Anayomy of a Spark Job on page 22, a shuffle associated with a wide dependency marks a new stage in the RDD's evaluation. Because tasks must be computed on a single partition and the data needed to compute each partition of a wide dependency may be spread across machines, transformations with wide dependencies may require data to be moved across partitions. Thus, the down-stream conputations cannot be computed before the shuffle finishes.

For example, it should be intuitive that sorting cannot be accomplished with narrow transformations be cause sorting requires an order to be defined on all of the records - not just within each partition. Indeed, the sorByKey function has wide dependencies. It reqruies the data to be partitioned, so all the keys within a certain range live on the same partition. That way, sorting the data on each partition leads to a sorted result. Any narrow transformations following the sort cannot be done until after the shuffle completes because the records on each partition may change.

## Implications for Fault Tolerance

The cost failure for a partition with dependencies is much higher than for one with narrow dependencies, since it requires more partitions to be recomputed. If one partition in the parent of a mappedRDD fails, for example, only one of its children must be recomputed, and the tasks needed to recompute that child partition can be distributed across the executors to make this recomputation faster. In contrast, if the parent of the sorted RDD loses a partition, it is possible that all the child partitions wil need to be recomputed. For this reason, the cost of recomputing a partition in the case of failure for a partition with wide dependencies is much higher than for a partition with narrow dependenceis.

## What Type of RDD Does your Transformation Return?

RDDs are an abstracted concept in two ways: they can b e of almost any arbitrary type of record, and they can also be members of one of several implementations of the RDD interface with varying properties. Both distinctions are important for performance and evaluation. The first is importatn, because some transformatiosn can only be applied to RDDs with certain record types. The second is important because each transformation returns one of the several implementations of the RDD interface, and therefore the same transformation called on two different RDD implementations may 

## Iterator-to-Iterator Transfomations with mapPartitions

The RDD mapPartitions function takes as its argument a function from an iterator of records to another iterator of records.

## Space and Time Advantages

The primary advantage of using iterator-to-iterator transformations in Spark routines is that their transformations allow Spark to selectively spill data to disk. Conceptually, an iterator-to-iterator transformation means defining a process for evaluating elements one at a time. Thus, Spark can apply that procedure to batches of records rather than reading an entire partition into memory or creating a collection with all of the output records in-memory and then returning it. Consequently, iterator-to-iterator transformations allow Spark to manipulate parititons that are too large to fit in memory on a single executor without out memory errors.

Furthermore, keeping the partition as an iterator allows Spark to use disk space more selectively. Rather than spilling an entire partition when it doesn't fit in memory, the iterator-to-iterator transformation allows Spark to spill only those records that do not fit in memory, thereby saving disk I/O and the cost of recomputation. Lastly, using methods defined on iterators avoids defining intermediary data structures. Reducing the number of large intermediate data structures is a way to avoid unnecessary object creation, which can slow down garbage collection as we talked about in.

# Working with key/value data